In [1]:
import os
import matplotlib.pyplot as plt
import ants
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# BASE_DIR tanımlama ve kontrol
BASE_DIR = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))), 'assets')
print(f'Assets folder = {BASE_DIR}')

# Dataset-S içinde AD veri seti yolu
dataset_path = os.path.join(BASE_DIR, 'testCNregister')
print(f'AD folder path = {dataset_path}')

# Klasörün varlığını kontrol et
if not os.path.exists(dataset_path):
    print(f"ERROR: Directory {dataset_path} does not exist!")
    print("\nAvailable directories:", os.listdir(BASE_DIR))
    exit()

# 📌 .nii dosyalarını listele
nii_files = [f for f in os.listdir(dataset_path) if f.endswith('.nii')]
print(f'Number of .nii files found: {len(nii_files)}')
if not nii_files:
    print("ERROR: No .nii files found in the dataset directory.")
    exit()

# 3D görüntüyü interaktif olarak görselleştiren fonksiyon
def explore_3D_array_fixed(arr, title=""):
    """
    3D NIfTI görüntüsünü interaktif görüntüle (dikey ters çevrilmiş şekilde).
    """
    fig_output = widgets.Output()
    
    def view_slice(slice_idx):
        with fig_output:
            clear_output(wait=True)
            plt.figure(figsize=(8, 8))
            flipped_slice = np.flipud(arr[:, :, slice_idx])  # 🔁 Dikey ters çevirme
            plt.imshow(flipped_slice, cmap='gray', aspect='equal')  # ✅ aspect='equal' eklendi
            plt.title(f"{title} - Slice {slice_idx}/{arr.shape[2]-1}")
            plt.axis('off')
            plt.show()  # ❌ plt.tight_layout() kaldırıldı
    
    # İlk slice'ı göster
    view_slice(0)
    
    # Kaydırıcı widget'ı
    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=arr.shape[2]-1,
        step=1,
        description='Slice:',
        continuous_update=False,
        layout=widgets.Layout(width='80%')
    )
    
    # Kaydırıcı ile görüntüleme fonksiyonunu bağla
    interactive_plot = widgets.interactive(view_slice, slice_idx=slider)
    
    # Tüm widget'ları göster
    return widgets.VBox([slider, fig_output])

# Global değişken olarak son görüntüyü sakla
last_loaded_image = None

# Dosya değiştiğinde ne yapılacağını tanımla
main_output = widgets.Output()

def load_file(file_name):
    global last_loaded_image
    with main_output:
        clear_output(wait=True)
        raw_img_path = os.path.join(dataset_path, file_name)
        print(f"\nProcessing file: {file_name}")
        # ANTs ile oku
        raw_img_ants = ants.image_read(raw_img_path, reorient='IAL')
        last_loaded_image = raw_img_ants.numpy()
        # Dosya bilgilerini göster
        print(f"Shape: {last_loaded_image.shape}")
        # İnteraktif görüntüleyici göster
        display(explore_3D_array_fixed(last_loaded_image, title=file_name))

# İleri ve geri butonları oluşturma
def create_navigation_buttons():
    prev_button = widgets.Button(
        description='⬅️ Önceki',
        button_style='info',
        tooltip='Önceki dosyaya git',
        layout=widgets.Layout(width='120px')
    )
    
    next_button = widgets.Button(
        description='Sonraki ➡️',
        button_style='info',
        tooltip='Sonraki dosyaya git',
        layout=widgets.Layout(width='120px')
    )
    
    def on_prev_button_clicked(b):
        current_index = nii_files.index(file_selector.value)
        new_index = (current_index - 1) % len(nii_files)
        file_selector.value = nii_files[new_index]
    
    def on_next_button_clicked(b):
        current_index = nii_files.index(file_selector.value)
        new_index = (current_index + 1) % len(nii_files)
        file_selector.value = nii_files[new_index]
    
    prev_button.on_click(on_prev_button_clicked)
    next_button.on_click(on_next_button_clicked)
    
    return widgets.HBox([prev_button, next_button])

# Dosya seçici dropdown oluşturma
file_selector = widgets.Dropdown(
    options=nii_files,
    description='Dosya seç:',
    layout=widgets.Layout(width='50%')
)

# Dosya seçicideki değişimleri dinleme
def on_file_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        load_file(change['new'])

file_selector.observe(on_file_change, names='value')

# Slice kaydedici fonksiyon ekleme
def save_selected_slices():
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    
    # Kayıt ana klasörünü tanımla
    base_save_folder = os.path.join(BASE_DIR, 'etiketlenecekler', 'AD')
    # Ana klasörü oluştur
    os.makedirs(base_save_folder, exist_ok=True)
    
    if last_loaded_image is not None:
        selected_slices = [128, 129, 130, 131, 132]
        current_file = file_selector.value
        base_filename = os.path.splitext(current_file)[0]  # .nii uzantısını kaldır
        
        # Her dosya için ayrı klasör oluştur
        file_save_folder = os.path.join(base_save_folder, base_filename)
        os.makedirs(file_save_folder, exist_ok=True)
        
        for slice_idx in selected_slices:
            slice_img = np.flipud(last_loaded_image[:, :, slice_idx])  # Dikey çevir
            save_path = os.path.join(file_save_folder, f"{base_filename}_slice_{slice_idx}.png")
            # PNG olarak kaydet
            plt.imsave(save_path, slice_img, cmap='gray')
        
        print(f"{len(selected_slices)} slice kaydedildi → {file_save_folder}")
    else:
        print("Henüz bir görüntü yüklenmedi.")

# Kaydetme butonu oluşturma
save_button = widgets.Button(
    description='Slice\'ları Kaydet',
    button_style='success',
    tooltip='Seçili görüntünün belirli slice\'larını kaydet',
    layout=widgets.Layout(width='150px')
)

save_button.on_click(lambda b: save_selected_slices())

# Widget'ları düzenle ve görüntüle
navigation_buttons = create_navigation_buttons()
controls = widgets.VBox([
    file_selector,
    widgets.HBox([navigation_buttons, save_button])
])
display(controls)
display(main_output)

# İlk dosyayı manuel olarak yükle
if nii_files:
    load_file(nii_files[0])

Assets folder = /app/assets
AD folder path = /app/assets/testCNregister
Number of .nii files found: 10


Output()

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Kayıt ana klasörünü tanımla
base_save_folder = os.path.join(BASE_DIR, 'etiketlenecekler', 'test','CN')
# Ana klasörü oluştur
os.makedirs(base_save_folder, exist_ok=True)

if last_loaded_image is not None:
    selected_slices =[125,126,127,128,129]
    current_file = file_selector.value
    base_filename = os.path.splitext(current_file)[0]  # .nii uzantısını kaldır
    
    # Her dosya için ayrı klasör oluştur
    file_save_folder = os.path.join(base_save_folder, base_filename)
    os.makedirs(file_save_folder, exist_ok=True)
    
    for slice_idx in selected_slices:
        slice_img = np.flipud(last_loaded_image[:, :, slice_idx])  # Dikey çevir
        save_path = os.path.join(file_save_folder, f"{base_filename}_slice_{slice_idx}.png")
        # PNG olarak kaydet
        plt.imsave(save_path, slice_img, cmap='gray')
    
    print(f"{len(selected_slices)} slice kaydedildi → {file_save_folder}")
else:
    print("Henüz bir görüntü yüklenmedi.")

5 slice kaydedildi → /app/assets/etiketlenecekler/test/CN/ADNI_941_S_1194_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20101111124352386_S80231_I204829_registered
